In [1]:
import csv
import numpy as np
import plotly
import plotly.tools as tls
from plotly.graph_objs import *
import os
import re

def get_csv_data(filepath):
    all_stats = {}
    apps = []
    data = {}
    any_data = False
    with open(filepath, 'r') as data_file:
        reader = csv.reader(data_file)        # define reader object
        state = "start"
        for row in reader:                    # loop through rows in csv file
            if len(row) != 0 and row[0].startswith("----"):
                state = "find-stat"
                continue
            if state == "find-stat":
                current_stat = row[0]
                state = "find-apps"
                continue
            if state == "find-apps":
                apps = [item.upper() for item in row[1:]]
                state = "process-cfgs"
                continue
            if state == "process-cfgs":
                if len(row) == 0:
                    if any_data:
                        all_stats[current_stat] = apps,data
                    apps = []
                    data = {}
                    state = "start"
                    any_data = False
                    continue
                temp = []
                for x in row[1:]:
                    try:
                        temp.append(float(x))
                        any_data = True
                    except ValueError:
                        temp.append(0)
                data[row[0]] = np.array(temp)

    return all_stats


In [ ]:
stats_map = {
    '\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)': 'L1 Total Reads',
    '\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)': 'L1 Read Hits',
    '\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)': 'L1 Read Miss',

    '\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)' : 'L1 Total TEX',
    '\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[HIT\]\s*=\s*(.*)' : 'L1 TEX Hits',
    '\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[HIT_RESERVED\]\s*=\s*(.*)' : 'L1 TEX Hits Resvered',

    '\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)' : 'L2 Total Reads',
    '\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)' : 'L2 Read Hits',

    'gpu_sim_cycle\s*=\s*(.*)' : 'Total Cycles',
    # 'L2_BW\s*=\s*(.*)GB\/Sec' : 'L2 BW'
}


this_directory = os.path.abspath(".")
concurrent = get_csv_data("../../per.kernel.stats.csv")
# delete_start = 14
# delete_end = 48
delete_start = 1<<31
delete_end = 1<<31

uids = []
for stat,value in concurrent.items():
    print(stat)
    if not "uid" in stat:
        continue
    apps, data = value
    uids = data["ORIN-SASS"]
uids = np.array(uids).astype(int)
uids = np.delete(uids,np.s_[delete_start:delete_end]).tolist()
uids = [str(x) for x in uids]

colors= ['#0F8C79','#BD2D28','#E3BA22','#E6842A','#137B80','#8E6C8A','#9A3E25', '#E6842A']
stat_count = 0
for stat,value in concurrent.items():
    if stat not in stats_map:
        continue
    stat = stats_map[stat]
    traces = []
    traces_abs = []
    cfg_count = 0
    apps, data = value
    base = np.array(data["ORIN-SASS"])
    base = np.delete(base,np.s_[delete_start:delete_end])
    print(data["ORIN-SASS"])
    for k,v in data.items():
        print(cfg_count)
        
        traces_abs.append(Bar(
            x= uids,
            y= v,
            name= k,
            marker=Marker(color=colors[cfg_count % len(colors)]),
            xaxis='x1',
            )
        )

        if k != "ORIN-SASS":
            v = np.array(v)
            v = np.delete(v,np.s_[delete_start:delete_end])
            to_plot = np.round(v / base,2)
            traces.append(Bar(
                x= uids,
                y= to_plot,
                name=k,
                marker=Marker(color=colors[cfg_count % len(colors)]),
                xaxis='x1', 
                text = to_plot
                )
            )
        cfg_count += 1

    data = Data(traces)
    data_abs = Data(traces_abs)
    layout = Layout(
        title=stat,
        barmode='group',
        bargroupgap=0,
        bargap=0.25,
        showlegend=True,
        yaxis=YAxis(
            title=stat,
        )
    )
    fig = Figure(data=data, layout=layout)
    fig.update_layout(yaxis_range=[0,3])
    fig.update_traces(textangle=0, cliponaxis=True)
    figure_name = re.sub('[^0-9a-zA-Z]+','_',stat)
    print("plotting: " + figure_name)
    outdir = (os.path.join(this_directory,"htmls"))
    if not os.path.exists( outdir ):
        os.makedirs(outdir)
    fig.show()
    # plotly.offline.plot(fig, filename=os.path.join(outdir,figure_name + ".html"),auto_open=False)

    fig = Figure(data=data_abs, layout=layout)
    fig.update_layout(yaxis_range=[0,3])
    fig.update_traces(textangle=0, cliponaxis=True)
    figure_name = re.sub('[^0-9a-zA-Z]+','_',stat)
    print("plotting: " + figure_name)
    outdir = (os.path.join(this_directory,"htmls"))
    if not os.path.exists( outdir ):
        os.makedirs(outdir)
    fig.show()
    # plotly.offline.plot(fig, filename=os.path.join(outdir,figure_name + ".html"),auto_open=False)

In [9]:
configs = ''
configs += "ORIN-SASS,"
configs += "ORIN-SASS-concurrent,"
for i in range(7,16):
    configs += "ORIN-SASS-concurrent-" + str(i) +'GR' + ','


configs = configs[:-1]
print(configs)

ORIN-SASS,ORIN-SASS-concurrent,ORIN-SASS-concurrent-7GR,ORIN-SASS-concurrent-8GR,ORIN-SASS-concurrent-9GR,ORIN-SASS-concurrent-10GR,ORIN-SASS-concurrent-11GR,ORIN-SASS-concurrent-12GR,ORIN-SASS-concurrent-13GR,ORIN-SASS-concurrent-14GR,ORIN-SASS-concurrent-15GR


In [7]:
import os
this_directory = os.path.abspath(".")
dir = os.getenv("HOME") + "/accel-sim-framework"
os.chdir(dir)
os.system("./util/job_launching/get_stats.py -R -B vulkan -C " + configs +" | tee per.kernel.stats.csv")

os.chdir(this_directory)